In [1]:
import pandas as pd
import numpy as np
import numpy.random as npr
from datetime import datetime, timedelta
from pyproj import Transformer
from concurrent.futures import ProcessPoolExecutor
import concurrent.futures
import multiprocessing
from functools import partial
import copy

import sys
import os
core_dir = os.path.join(os.getcwd(), '..')
core_dir = os.path.normpath(core_dir)
sys.path.append(core_dir)
core_dir = os.path.join(os.getcwd(), '..', 'daphme')
core_dir = os.path.normpath(core_dir)
sys.path.append(core_dir)

import garden_city as gcity
from garden_city import Building, Street, City, Agent, Population
import mobility_model as mmod
import stop_detection as sd
from constants import DEFAULT_SPEEDS, FAST_SPEEDS, SLOW_SPEEDS, DEFAULT_STILL_PROBS, FAST_STILL_PROBS, SLOW_STILL_PROBS

import pdb

In [2]:
pd.options.mode.chained_assignment = None

### Generate N agents

In [4]:
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)


def garden_city_to_lat_long(agent):
    df = agent.sparse_traj
    df['x'] = 15*df['x'] + 4265699
    df['y'] = 15*df['y'] - 4392976

    df['latitude'], df['longitude'] = transformer.transform(
        df['x'].values, df['y'].values)

    df = df[['identifier', 'unix_timestamp', 'latitude', 'longitude']]
    df = df.rename(columns={'identifier': 'uid', 'unix_timestamp': 'timestamp'})
    df = df.reset_index(drop=True)

    agent.sparse_traj = df

    df = agent.trajectory
    df['x'] = 15*df['x'] + 4265699
    df['y'] = 15*df['y'] - 4392976

    df['latitude'], df['longitude'] = transformer.transform(
        df['x'].values, df['y'].values)

    df = df[['identifier', 'unix_timestamp', 'latitude', 'longitude']]
    df = df.rename(columns={'identifier': 'uid', 'unix_timestamp': 'timestamp'})
    df = df.reset_index(drop=True)

    agent.trajectory = df
    return None

In [5]:
%%time

npr.seed(100)

city = gcity.load('garden-city.pkl')
population_n = Population(city)
population_n.generate_agents(1000,
                             start_time=datetime(2024, 1, 1, hour=8, minute=0),
                             dt=1, seed=123)

def generate_agent_trajectory(agent_id, agent, seed):
    population_n.generate_trajectory(agent, T=datetime(2024, 1, 15, hour=8, minute=0), seed=seed)
    
    # THOMAS: I choose these values quickly, might want to put more thought into the ranges
    beta_start = npr.uniform(60, 1200)
    beta_duration = npr.uniform(15, 180)
    beta_ping = npr.uniform(5, 30)
    
    param = (beta_start, beta_duration, beta_ping)
    agent.sample_traj_hier_nhpp(*param, seed=seed)
    garden_city_to_lat_long(agent)
    
    return agent_id, copy.deepcopy(agent)

manager = multiprocessing.Manager()
shared_roster = manager.dict(population_n.roster)

with ProcessPoolExecutor() as executor:
    futures = [
        executor.submit(generate_agent_trajectory, agent_id, agent, i)
        for i, (agent_id, agent) in enumerate(shared_roster.items())
    ]
    for future in futures:
        agent_id, agent = future.result()
        shared_roster[agent_id] = agent

population_n.roster = dict(shared_roster)

population_n.save_pop(bucket='synthetic-raw-data', prefix='1000-agents/')

CPU times: user 6min 22s, sys: 24.7 s, total: 6min 46s
Wall time: 17min 39s


In [6]:
population_n.roster['agitated_chebyshev'].trajectory

,uid,timestamp,latitude,longitude
0,agitated_chebyshev,1704096000,38.321380,-36.666867
1,agitated_chebyshev,1704096060,38.321456,-36.666672
2,agitated_chebyshev,1704096120,38.320961,-36.666614
3,agitated_chebyshev,1704096180,38.320490,-36.666651
4,agitated_chebyshev,1704096240,38.320305,-36.666932
...,...,...,...,...
20156,agitated_chebyshev,1705305360,38.320429,-36.666991
20157,agitated_chebyshev,1705305420,38.320429,-36.666991
20158,agitated_chebyshev,1705305480,38.320429,-36.666991
20159,agitated_chebyshev,1705305540,38.320429,-36.666991
